<a href="https://colab.research.google.com/github/Yusmitha-Lekha/YusmithaLekha_INFO5731_Fall2024/blob/main/YusmithaLekhaPrathi_Dataset5_RHMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv

# Input and output file paths
input_file = 'RHMD_3_Class.csv'  # Replace with your actual input CSV file
output_file = 'output_file.csv'  # Replace with your desired output CSV file

# Define a dictionary to map numeric labels to their descriptions
label_map = {
    '0': 'Figurative Mentions',
    '1': 'Other Mentions',
    '2': 'Health mentions'
}

# Open the input CSV file
with open(input_file, 'r') as csv_file:
    reader = csv.reader(csv_file)

    # Read the rows
    rows = list(reader)

# Open the output CSV file to write the updated content
with open(output_file, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Write the header row to the output CSV file (assuming you have headers)
    writer.writerow(rows[0])

    # Iterate through the rows (skipping the header)
    for row in rows[1:]:  # Skip the header row
        if len(row) == 2:  # Ensure the row has exactly 2 columns: Text, Label
            # Replace the numeric label with its description
            row[1] = label_map.get(row[1], row[1])  # Replace label if found in the dictionary

            # Write the updated row to the output CSV file
            writer.writerow(row)
        else:
            print(f"Skipping invalid row: {row}")  # Optional: Print the row that's being skipped

print(f"File '{input_file}' has been updated with labels and saved as '{output_file}'.")


File 'RHMD_3_Class.csv' has been updated with labels and saved as 'your_output_file.csv'.


In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import re

# Load the Hugging Face pipeline for Zero-Shot classification
zero_shot_classifier = pipeline("zero-shot-classification")

# Function to get Hugging Face response for Zero-Shot classification
def get_huggingface_response(prompt, model="facebook/bart-large-mnli", temperature=0.7):
    result = zero_shot_classifier(prompt, candidate_labels=["correct", "incorrect", "complete", "incomplete", "comprehensive", "incomplete"])
    return result['scores'][0]  # Return the score for the most likely label

# Function to compute correctness score using Hugging Face (Zero-Shot and Few-Shot)
def compute_correctness_score(description, disease_term, model="facebook/bart-large-mnli"):
    # Zero-shot Prompt: Hugging Face should evaluate how similar the disease term is to the description
    zero_shot_prompt = f"Evaluate the correctness of the disease description in relation to the disease term. The description: '{description}' and the disease term: '{disease_term}'. Rate the correctness on a scale from 0 to 1."

    correctness_score_zero_shot = get_huggingface_response(zero_shot_prompt, model)
    return correctness_score_zero_shot


# Function to compute completeness score using Hugging Face (Zero-Shot and Few-Shot)
def compute_completeness_score(description, model="facebook/bart-large-mnli"):
    # Zero-shot Prompt: Hugging Face should evaluate if the description covers all aspects of the disease
    zero_shot_prompt = f"Is the following disease description complete? Does it include key symptoms, diagnostic information, and relevant details about the disease? Describe any missing aspects. The description: '{description}'"

    completeness_score_zero_shot = get_huggingface_response(zero_shot_prompt, model)
    return completeness_score_zero_shot


# Function to compute comprehensiveness score using Hugging Face (Zero-Shot and Few-Shot)
def compute_comprehensiveness_score(description, model="facebook/bart-large-mnli"):
    # Zero-shot Prompt: Hugging Face should evaluate the comprehensiveness of the description
    zero_shot_prompt = f"Is the following disease description comprehensive? Does it include symptoms, diagnostic tests, treatments, and all relevant details? Rate the comprehensiveness on a scale from 0 to 1. The description: '{description}'"

    comprehensiveness_score_zero_shot = get_huggingface_response(zero_shot_prompt, model)
    return comprehensiveness_score_zero_shot


# Few-shot Prompting Example for Correctness, Completeness, and Comprehensiveness
def few_shot_prompt(prompt, model="gpt2"):
    # Example 1: Correctness
    example_1 = "Description: 'A condition characterized by joint pain and stiffness. Often causes inflammation.' Disease Term: 'Arthritis'. Correctness: 0.8"
    example_2 = "Description: 'A disorder affecting the nervous system with symptoms of paralysis and muscle weakness.' Disease Term: 'Multiple Sclerosis'. Correctness: 0.9"

    # Example 2: Completeness
    example_3 = "Description: 'A disease caused by the flu virus, characterized by fever, chills, sore throat, and fatigue.' Completeness: 0.8"
    example_4 = "Description: 'A chronic condition involving high blood sugar levels and insulin resistance.' Completeness: 0.9"

    # Example 3: Comprehensiveness
    example_5 = "Description: 'A condition that involves an autoimmune response that attacks the joints.' Comprehensiveness: 0.7"
    example_6 = "Description: 'A mental health condition characterized by persistent sadness, feelings of hopelessness, and a lack of interest in daily activities.' Comprehensiveness: 0.85"

    # Combine the examples and prompt
    few_shot_examples = "\n".join([example_1, example_2, example_3, example_4, example_5, example_6])

    prompt_with_few_shot = f"{few_shot_examples}\n\n{prompt}"

    # Use the GPT-2 model from Hugging Face for Few-Shot Prompting
    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModelForCausalLM.from_pretrained(model)

    inputs = tokenizer(prompt_with_few_shot, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=400, num_return_sequences=1, temperature=0.7)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the numerical score from the response
    match = re.search(r"(\d+\.\d+)", response)
    if match:
        return float(match.group(1))  # Return the extracted score
    else:
        return 0.0  # Return a default score in case no score is found


# Read the CSV file using pandas (replace 'your_file.csv' with the actual file path)
csv_file_path = 'output_file.csv'  # Replace this with your actual CSV file path
df = pd.read_csv(csv_file_path)

# Initialize variables to accumulate scores for averaging
zero_shot_correctness_total = 0
zero_shot_completeness_total = 0
zero_shot_comprehensiveness_total = 0

few_shot_correctness_total = 0
few_shot_completeness_total = 0
few_shot_comprehensiveness_total = 0

# Evaluate the first few rows of the CSV file
num_rows = 7
for i in range(num_rows):
    description = df.iloc[i]['Text']  # Adjust the column name accordingly
    disease_term = df.iloc[i]['Label']  # Adjust the column name accordingly

    # Compute the scores using Zero-Shot and Few-Shot approaches

    # Zero-Shot Approach
    correctness_score_zero_shot = compute_correctness_score(description, disease_term)
    completeness_score_zero_shot = compute_completeness_score(description)
    comprehensiveness_score_zero_shot = compute_comprehensiveness_score(description)

    zero_shot_correctness_total += correctness_score_zero_shot
    zero_shot_completeness_total += completeness_score_zero_shot
    zero_shot_comprehensiveness_total += comprehensiveness_score_zero_shot

    print(f"Row {i+1} - Zero-Shot Correctness Score: {correctness_score_zero_shot}")
    print(f"Row {i+1} - Zero-Shot Completeness Score: {completeness_score_zero_shot}")
    print(f"Row {i+1} - Zero-Shot Comprehensiveness Score: {comprehensiveness_score_zero_shot}")

    # Few-Shot Approach
    few_shot_correctness_prompt = f"Evaluate the correctness of the disease description: '{description}' with the disease term: '{disease_term}'. Rate the correctness from 0 to 1."
    few_shot_completeness_prompt = f"Is the following disease description complete? Rate the completeness from 0 to 1. The description: '{description}'"
    few_shot_comprehensiveness_prompt = f"Evaluate if the description includes symptoms, diagnostic tests, and treatments. Rate the comprehensiveness from 0 to 1. The description: '{description}'"

    correctness_score_few_shot = few_shot_prompt(few_shot_correctness_prompt)
    completeness_score_few_shot = few_shot_prompt(few_shot_completeness_prompt)
    comprehensiveness_score_few_shot = few_shot_prompt(few_shot_comprehensiveness_prompt)

    few_shot_correctness_total += correctness_score_few_shot
    few_shot_completeness_total += completeness_score_few_shot
    few_shot_comprehensiveness_total += comprehensiveness_score_few_shot

    print(f"Row {i+1} - Few-Shot Correctness Score: {correctness_score_few_shot}")
    print(f"Row {i+1} - Few-Shot Completeness Score: {completeness_score_few_shot}")
    print(f"Row {i+1} - Few-Shot Comprehensiveness Score: {comprehensiveness_score_few_shot}")

# Calculate and print the average scores for Zero-Shot and Few-Shot
zero_shot_correctness_avg = zero_shot_correctness_total / num_rows
zero_shot_completeness_avg = zero_shot_completeness_total / num_rows
zero_shot_comprehensiveness_avg = zero_shot_comprehensiveness_total / num_rows

few_shot_correctness_avg = few_shot_correctness_total / num_rows
few_shot_completeness_avg = few_shot_completeness_total / num_rows
few_shot_comprehensiveness_avg = few_shot_comprehensiveness_total / num_rows

print(f"\nZero-Shot Average Correctness Score: {zero_shot_correctness_avg}")
print(f"Zero-Shot Average Completeness Score: {zero_shot_completeness_avg}")
print(f"Zero-Shot Average Comprehensiveness Score: {zero_shot_comprehensiveness_avg}")

print(f"\nFew-Shot Average Correctness Score: {few_shot_correctness_avg}")
print(f"Few-Shot Average Completeness Score: {few_shot_completeness_avg}")
print(f"Few-Shot Average Comprehensiveness Score: {few_shot_comprehensiveness_avg}")


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Row 1 - Zero-Shot Correctness Score: 0.3121579587459564
Row 1 - Zero-Shot Completeness Score: 0.42948636412620544
Row 1 - Zero-Shot Comprehensiveness Score: 0.3438677191734314


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Row 5 - Few-Shot Correctness Score: 0.8
Row 5 - Few-Shot Completeness Score: 0.8
Row 5 - Few-Shot Comprehensiveness Score: 0.8
Row 6 - Zero-Shot Correctness Score: 0.28105273842811584
Row 6 - Zero-Shot Completeness Score: 0.44147440791130066
Row 6 - Zero-Shot Comprehensiveness Score: 0.35733476281166077


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Row 6 - Few-Shot Correctness Score: 0.8
Row 6 - Few-Shot Completeness Score: 0.8
Row 6 - Few-Shot Comprehensiveness Score: 0.8
Row 7 - Zero-Shot Correctness Score: 0.28440532088279724
Row 7 - Zero-Shot Completeness Score: 0.43124085664749146
Row 7 - Zero-Shot Comprehensiveness Score: 0.3526807725429535


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Row 7 - Few-Shot Correctness Score: 0.8
Row 7 - Few-Shot Completeness Score: 0.8
Row 7 - Few-Shot Comprehensiveness Score: 0.8

Zero-Shot Average Correctness Score: 0.2844099743025644
Zero-Shot Average Completeness Score: 0.43996440512793406
Zero-Shot Average Comprehensiveness Score: 0.3486120913709913

Few-Shot Average Correctness Score: 0.7999999999999999
Few-Shot Average Completeness Score: 0.7999999999999999
Few-Shot Average Comprehensiveness Score: 0.7999999999999999


In [ ]:
import pandas as pd
from tqdm import tqdm

# Function to extract unique pairs of disease and description
def get_unique_pairs(df):
    unique_pairs = set()  # Store unique pairs in a set to avoid duplicates
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing Entries"):
        pair = (row['Label'], row['Text'])
        unique_pairs.add(pair)
    return unique_pairs

# Function to calculate uniqueness score for a dataset
def calculate_uniqueness_score(df):
    unique_pairs = get_unique_pairs(df)
    total_entries = len(df)
    unique_pairs_count = len(unique_pairs)
    uniqueness_score = unique_pairs_count / total_entries
    return uniqueness_score

# Load the dataset
dataset_path = 'output_file.csv'
df = pd.read_csv(dataset_path)

# Calculate and print the uniqueness score for the dataset
uniqueness_score = calculate_uniqueness_score(df)
print(f"Uniqueness Score {uniqueness_score:.4f}")


Processing Entries: 100%|██████████| 10015/10015 [00:01<00:00, 10005.07it/s]

Uniqueness Score 0.9813
